In [52]:
# Tokens for connection to different services
from clan_tools.secrets.Vault import Vault
Vault().get_secrets()

# Connector to YT by ClickHouse
from clan_tools.data_adapters.ClickHouseYTAdapter import ClickHouseYTAdapter
ch_adapter = ClickHouseYTAdapter()

# Connector to YT
from clan_tools.data_adapters.YTAdapter import YTAdapter
yt_adapter = YTAdapter()

import numpy as np
import itertools as itert
import pandas as pd

In [72]:
dm_yc_cons_path = '\"//home/cloud-dwh/data/prod/cdm/dm_yc_consumption\"'
ba_id_with_service_name_path = '\"//home/cloud_analytics/data_swamp/projects/service_groups_penetration/ba_with_service_name_20220101_wo_c\"'
# Used ((https://yql.yandex-team.ru/Operations/Ygvxe9JwbE55RtLyWrOm9iknGXa0klUB7D2abClyp7Q= this)) to make ba_id_with_service_name 
service_name_sql = '''
    if(
        sku_service_name in ('cloud_ai', 'mdb'),
        concat(sku_service_name, ' - ', sku_subservice_name),
        sku_service_name
    )
'''

In [70]:
service_name_sql = '''
    select
        {service_name} as service_name
    from
        {table_path}
    where
        sku_service_group not in ('Adjustments', 'Support')
    group by
        service_name
'''.format(table_path=dm_yc_cons_path, service_name=service_name_sql)

service_name_arr = ch_adapter.execute_query(query=service_name_sql, to_pandas=True).service_name.values
service_name_arr = service_name_arr[service_name_arr != None]

In [71]:
print(service_name_arr)
print(len(service_name_arr))

['storage' 'mdb - dataproc' 'cloud_ai - mt' 'cloud_ai - subscription'
 'mdb - postgres' 'managed-gitlab' 'kms' 'cloud_ai - vision' 'yds'
 'cloud_ai - locator' 'mdb - network_ssd' 'mdb - mysql' 'cloud_network'
 'mk8s' 'mdb - clickhouse' 'ymq' 'dns' 'mdb - mssql' 'datalens'
 'monitoring' 'api-gateway' 'professional-services' 'mdb - public_ip'
 'mdb - redis' 'cr' 'tracker' 'cloud_ai - datasphere' 'alb'
 'cloud_ai - adjustment' 'cloud_ai - speech' 'ydb' 'mdb - kafka'
 'marketplace' 'nlb' 'mdb - local_ssd' 'serverless' 'compute' 'iot' 'cdn'
 'mdb - greenplum' 'mdb - mongo' 'mdb - elasticsearch']
42


In [66]:

popular_service_combination_df = pd.DataFrame(columns=['service_name', 'ba_count'])
popular_service_name_combination = ''

for service_count in range(len(service_name_arr)):
# for service_count in range(10):
    service_name_combination_sql = '''
        select
            concat('{popular_service_name}', ',', service_name) as service_name_combination
        from(
            select
                {service_name} as service_name
            from
                {table_path}
            where
                sku_service_group not in ('Adjustments', 'Support')
            group by
                service_name
        )
        where
            position('{popular_service_name}', service_name) = 0
    '''.format(table_path=dm_yc_cons_path, popular_service_name=popular_service_name_combination, service_name=service_name_sql)

    popular_service_name_sql = '''
        select
            argMax(service_name_combination, ba_c) as service_name,
            max(ba_c) as ba_count
        from(
            select
                service_name_combination,
                count(distinct
                    if(
                        position(service_name_combination, service_name) != 0,
                        billing_account_id,
                        '-1'
                    )
                ) - 1 as ba_c
            from {ba_id_with_service_name} as ba_id
                cross join ({service_name_combination}) as combination
                group by
                    service_name_combination
        )
    '''.format(
        table_path=dm_yc_cons_path, 
        service_name_combination=service_name_combination_sql, 
        service_name=service_name_sql, 
        ba_id_with_service_name=ba_id_with_service_name_path
    )
# print(popular_service_name_sql)

    new_popular_service_name_df = ch_adapter.execute_query(query=popular_service_name_sql, to_pandas=True)
    popular_service_name_combination = new_popular_service_name_df.service_name.values[0]
    popular_service_combination_df = pd.concat([popular_service_combination_df, new_popular_service_name_df], ignore_index=True)


In [68]:
popular_service_combination_df_total = popular_service_combination_df